
# Discord Activities



## Client Code



### Override Links

In Discord client, the links have to use window.parent.postMessage in order to open external links otherwise the client closes on mobile, this is to prevent the client from being used in phishing attempts redirecting users to nefarious websites. Instead all web traffic goes through Discord's servers.




#### the code

discord client code?


In [ ]:

let code
let userId

function sourceWindow() {
  return (window.parent.opener || window.parent)
}

document.addEventListener('click', async (evt) => {
  if(evt.target.tagName == 'A' && evt.target.getAttribute('href').includes('://')) {
    debugger


    sourceWindow().postMessage([1, {
      cmd: 'OPEN_EXTERNAL_LINK', 
      args: {url: evt.target.href}, 
      nonce: Math.random() * 1000,
      transfer: void 0
    }], '*')

    evt.preventDefault()
    return false
  }

  if(evt.target.tagName == 'A' && evt.target.getAttribute('href').includes('/.proxy/')) {
    const loggedInResponse = await fetch(evt.target.getAttribute('href'))
    const newContent = await loggedInResponse.text()
    //if(!newContent.includes('Cannot GET'))
    document.body.innerHTML = newContent.replace(/<\/?html>/gi, '')
                                        .replace(/<script.*?>[\s\S]*?<\/script>/gi, '');
  }
  return true
})

let result = {}
let params = (window.location.search.split('?')[1] || '').split('&')
for(var param in params) {
  if (params.hasOwnProperty(param)) {
    let paramParts = params[param].split('=')
    result[paramParts[0]] = decodeURIComponent(paramParts[1] || "")
  }
}

document.addEventListener("DOMContentLoaded", (event) => {

  debugger

  if(typeof result.frame_id == 'undefined') {
    return
  }

})



### Authorize Code

Mostly from the documentation and watching another successful app make the connection. Basic steps are say hello, dispatch events, authorize, then authenticate. Because it's using my server as a token proxy, I'm also validating the user_id server side and offering logged in users extra features.




#### the code

discord client auth code?


In [ ]:

let expiry
let result
let initNonce = Math.round(Math.random() * 1000)

window.addEventListener('message', async (evt) => {

  document.querySelector('h1').innerHTML = '<pre>' + JSON.stringify(evt.data, null, 2) + '</pre>'

  const sourceWindow = () => {
    return (window.parent.opener || window.parent)
  }

  if(evt.data[0] == 3 && typeof evt.data[1].frame_id != 'undefined') {

    result = evt.data[1] // get it from page init instead of query params
    // send handshake
    sourceWindow().postMessage([0, {
      v: 1,
      encoding: 'json',
      client_id: '{CLIENT_ID}',
      frame_id: evt.data[1].frame_id,
      nonce: initNonce,
      transfer: void 0
    }], 'https://discord.com' || '*')
  }

  if(evt.data[0] == 1 && evt.data[1].cmd == 'DISPATCH') {

    // start authorization chain
    sourceWindow().postMessage([1, {
      cmd: 'AUTHORIZE',
      args: {
        client_id: '{CLIENT_ID}',
        prompt: 'none',
        response_type: 'code',
        scope: ['applications.commands', 'identify', 'guilds.members.read', 'rpc.activities.write'],
        state: '',
      },
      nonce: Math.random() * 1000,
      transfer: void 0
    }], 'https://discord.com' || '*')

  }

  if(evt.data[1].cmd == 'AUTHORIZE') {
    code = evt.data[1].data.code

    const response = await fetch("{BASE_URI}api/token", {
      method: "POST",
      headers: {
        "Content-Type": "application/json",
      },
      credentials: 'same-origin',
      body: JSON.stringify({
        code,
        launch_id: result.launch_id,
        frame_id: result.frame_id,
      }),
    });
    const { access_token, expires_in, session_id } = await response.json();
    window.session_id = session_id
    
    debugger

    // TODO: take authenticate commands
  
    sourceWindow().postMessage([1, {
      cmd: 'AUTHENTICATE',
      args: {
        access_token: access_token
      },
      nonce: Math.random() * 1000,
      transfer: void 0
    }], 'https://discord.com' || '*')

    expiry = Date.now() + expires_in
  }

  if(evt.data[1].cmd == 'AUTHENTICATE') {
    // TODO: some authenticated page update
    // trying to solve the cookie problem
    if(window.session_id)
      window.location = '{BASE_URI}?t=' + Date.now() + '&session=' + window.session_id
  }
})




## Activity Server




### Start Express

this should be replaced by code in discord remote and discord player with the correct routes handled.



#### the code

start activity server?


In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
//const {discordExpress, closeExpress, getInstance} = importer.import('discord express server')

const ALL_COMMANDS = [
  'launch',
]

// bot commands using new API, same names as above but lower-case
async function activityCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress

  cmdDef = {
    'name': 'launch',
    'description': 'Launch an activity',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('launch')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'launch')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = activityCommands




check discord commands?


In [ ]:

const {getCommands} = importer.import('discord api')

module.exports = getCommands



### All Bots

A list of a bot start ups with formulated environments and launch commands.



#### the code

start a bunch of discord services?


In [ ]:
const {spawn, spawnSync} = require("child_process");
const path = require('path')

const DISCORD_SERVICES = [
  void 0,
  void 0, // TODO: Orbb bot
  'discord mid journey commands',
  'sync deceptive chat',
  'sync discord llm tools',
  'discord handy tools',
  'discord remote control',
  'discord music player',
]

const APPLICATION_IDS = [
  void 0,
  void 0,
  '1335464182111535124',
  '1335472379232780401',
  '1335479586858733568',
  '1335483072602177626',
  '1335491680630997034',
  '1335769252409380884',
]

const ENVIRONMENTS = [
  void 0,
  void 0,
  void 0,
  void 0,
  void 0,
  void 0,
  void 0,
]

function launchDiscord(botIndex = 0) {

  if(parseInt(botIndex).toString() != botIndex.toString()) {
    return
  } else {
    botIndex = parseInt(botIndex)
  }

  if(typeof botIndex != 'number' && !botIndex) {
    return
  }

  for(let i = botIndex; i < DISCORD_SERVICES.length; i++) {
    if(!DISCORD_SERVICES[i]) {
      continue
    }

    spawn('node', ['--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', DISCORD_SERVICES[i]], {
      env: Object.assign({}, process.env, {
        TOKENPATH: '/Users/briancullinan/.credentials/discord-bot' + (i ? i : '') + '.txt',
        SECRETPATH: '/Users/briancullinan/.credentials/discord' + (i ? i : '') + '.txt',
        DEFAULT_APPLICATION: APPLICATION_IDS[i]
      }, ENVIRONMENTS[i] ? ENVIRONMENTS[i] : {}),
      stdio: [0, 1, 2],
      //detached: true,
      //shell: true,
      cwd: path.dirname(__dirname)
    })
  
    if(botIndex) {
      break
    }
  }
}

module.exports = launchDiscord



### Express Authenticator

Authenticate a page using the passed in session query parameter. This is vulnerable to man in the middle attacks because it goes through Discord proxy. Somebody knows.



#### the code

discord authenticate instances?


In [ ]:
const {INSTANCES, SESSIONS} = importer.import('discord gateway')

const ALLOWED_USERS = [
  '1019970191603544164'
]

async function authenticateRoute(req, res, next) {
  var url_parts = url.parse(req.url, true);
  var query = url_parts.query;
  //console.log(query, SESSIONS, query.session && typeof SESSIONS[query.session] != 'undefined' 
  //  && ALLOWED_USERS.includes(SESSIONS[query.session]))
  if(query.session && typeof SESSIONS[query.session] != 'undefined' 
    && ALLOWED_USERS.includes(SESSIONS[query.session])) {
    req.authenticated = query.session
    return next()
  }

  // TODO: check if the user is in fact apart of the activity
  req.authenticated = false

  return next()
}


module.exports = {
  //registerInstance,
  authenticateRoute,
  INSTANCES,
  SESSIONS,
  
}



### Token Checkout

Get a token from discord through this application. This is how the SECRET is kept secret from clients, but there's an option in Discord for that scenario too. This turns my application into a Discord JWT client within the webframe embedded inside the Discord app.



#### the code


discord express token endpoint?


In [ ]:
const {INSTANCES, SESSIONS} = importer.import('discord gateway')
const {request} = require('gaxios')
const qs = require('qs');
const {DEFAULT_API, SECRET, TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
var crypto = require('crypto')

async function getToken(req, res, next) {
  //console.log(req.body)
  let result = await request({
    url: DEFAULT_API + 'oauth2/token',
    method: 'POST',
    headers: {
      'Content-Type': 'application/x-www-form-urlencoded',
      //'Authorization': 'Bot ' + TOKEN
    },
    data: qs.stringify({
      client_id: DEFAULT_APPLICATION,
      client_secret: SECRET,
      grant_type: 'authorization_code',
			code: req.body.code,
		})
  })

  let expiry = new Date(Date.now() + result.data.expires_in)

  if(typeof req.body['launch_id'] != 'undefined' 
    && typeof req.body['frame_id'] != 'undefined')
  {
    let userResult = await request({
      method: 'GET',
      url: DEFAULT_API + `oauth2/@me`,
      headers: {
        'Authorization': 'Bearer ' + result.data.access_token
      }
    })

    // TODO: group permissions

    if(typeof INSTANCES[req.body['launch_id']] == 'undefined') {
      INSTANCES[req.body['launch_id']] = []
    }
    INSTANCES[req.body['launch_id']].push([userResult.data.user.id, req.body['frame_id']])
    console.log(INSTANCES)

    // generate nonce for client to use on future commands to authenticate with our server quickly instead of relying on cookies
    result.data.session_id = crypto.randomBytes(16).toString('hex')
    SESSIONS[result.data.session_id] = userResult.data.user.id

    res.cookie('discord_bot_session', userResult.data.user.id, {
      path: '/.proxy/',
      expires: expiry
    })
    res.cookie('discord_bot_frame', req.body['frame_id'], { 
      path: '/.proxy/',
      expires: expiry
    })
  }

  res.cookie('discord_bot_expiry', Date.now() + result.data.expires_in, { 
    path: '/.proxy/',
    expires: expiry
  })
  res.setHeader('Content-Type', 'application/json')
  return res.send(JSON.stringify(result.data))
}

module.exports = getToken
